In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import joblib
import datetime
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message="X does not have valid feature names")

# File paths
file_path = 'final_featured_stock_data_cleaned.csv'
save_dir = '/Users/admin/Downloads'
os.makedirs(save_dir, exist_ok=True)

# Timestamp for model and outputs
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model_path = os.path.join(save_dir, f'lstm_stock_N_model_{timestamp}.h5')
scaler_path = os.path.join(save_dir, f'stock_N_scalers_{timestamp}.pkl')

# Load data
print("Loading data...")
df = pd.read_csv(file_path)
df['Date'] = pd.to_datetime(df['Date'])

# Extract only Stock N data
stock_n_data = df[df['Stock_ID'] == 'Stock N'].copy()
print(f"Stock N data points: {len(stock_n_data)}")
print(f"Date range: {stock_n_data['Date'].min()} to {stock_n_data['Date'].max()}")

# Check for missing values in Stock N data
missing_values = stock_n_data.isna().sum()
print("Missing values in Stock N:")
print(missing_values[missing_values > 0])

# Handle missing values
for col in stock_n_data.columns:
    if stock_n_data[col].isna().sum() > 0:
        if col in ['SMA_10', 'EMA_10', 'RSI_14', 'MACD', 'Signal_Line', 'OBV', 'Rolling_Mean_20', 'Rolling_Std_20', 'Bollinger_Upper', 'Bollinger_Lower']:
            # For technical indicators, forward fill is appropriate
            stock_n_data[col] = stock_n_data[col].fillna(method='ffill')
            # For any remaining NaNs at the beginning, backward fill
            stock_n_data[col] = stock_n_data[col].fillna(method='bfill')
        else:
            # For lag and rolling features
            stock_n_data[col] = stock_n_data[col].fillna(method='ffill')
            stock_n_data[col] = stock_n_data[col].fillna(method='bfill')

# Verify all missing values are handled
if stock_n_data.isna().sum().sum() > 0:
    print("Remaining missing values after cleaning:")
    print(stock_n_data.isna().sum()[stock_n_data.isna().sum() > 0])
else:
    print("All missing values in Stock N data have been handled.")

# Select feature columns and target
exclude_cols = ['Date', 'Stock_ID', 'Year', 'Month', 'Day', 'Day_of_Week', 'Quarter', 'time_idx']
feature_cols = [col for col in stock_n_data.columns if col not in exclude_cols and col != 'Close']
target_col = 'Close'

print(f"Using {len(feature_cols)} features for Stock N model")

# Sort by date
stock_n_data = stock_n_data.sort_values('Date').reset_index(drop=True)

# Sequence parameters - adjusted for Stock N's smaller dataset
sequence_length = 10  # Shorter sequence due to limited data
prediction_days = 1   # Predict one day at a time

# Function to prepare data for LSTM
def prepare_data(data, feature_cols, target_col, sequence_length, prediction_days):
    """
    Prepare data for LSTM model
    
    Parameters:
    data (DataFrame): DataFrame containing stock data
    feature_cols (list): List of feature column names
    target_col (str): Target column name
    sequence_length (int): Number of past time steps to use
    prediction_days (int): Number of days to predict ahead
    
    Returns:
    tuple: (X_train, y_train, X_val, y_val, X_test, y_test, scalers)
    """
    # Create scalers for features and target
    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    target_scaler = MinMaxScaler(feature_range=(0, 1))
    
    # Fit and transform feature data
    feature_data = feature_scaler.fit_transform(data[feature_cols])
    
    # Fit and transform target data
    target_data = target_scaler.fit_transform(data[[target_col]])
    
    # Create sequences
    X, y = [], []
    for i in range(len(feature_data) - sequence_length - prediction_days + 1):
        X.append(feature_data[i:(i + sequence_length)])
        y.append(target_data[i + sequence_length + prediction_days - 1])
    
    X, y = np.array(X), np.array(y)
    
    # Split into train (70%), validation (15%), and test (15%)
    # For small datasets, we can use a higher percentage for training
    train_size = int(len(X) * 0.8)
    val_size = int(len(X) * 0.1)
    
    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
    X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]
    
    print(f"Training sequences: {len(X_train)}")
    print(f"Validation sequences: {len(X_val)}")
    print(f"Testing sequences: {len(X_test)}")
    
    return X_train, y_train, X_val, y_val, X_test, y_test, (feature_scaler, target_scaler)

# Function to build and train LSTM model
def build_lstm_model(X_train, y_train, X_val, y_val, save_path):
    """
    Build and train LSTM model
    
    Parameters:
    X_train (array): Training features
    y_train (array): Training target
    X_val (array): Validation features
    y_val (array): Validation target
    save_path (str): Path to save the model
    
    Returns:
    model: Trained LSTM model
    history: Training history
    """
    # Define model architecture - simpler for smaller dataset
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True
    )
    
    # Model checkpoint
    checkpoint = ModelCheckpoint(
        filepath=save_path,
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    
    # Train model - more epochs for smaller dataset
    history = model.fit(
        X_train, y_train,
        epochs=200,
        batch_size=8,  # Smaller batch size for smaller dataset
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, checkpoint],
        verbose=1
    )
    
    return model, history

# Function to evaluate model
def evaluate_model(model, X_test, y_test, target_scaler):
    """
    Evaluate model performance
    
    Parameters:
    model: Trained LSTM model
    X_test (array): Test features
    y_test (array): Test target
    target_scaler: Scaler used for target variable
    
    Returns:
    dict: Performance metrics
    """
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Inverse transform predictions and actual values
    y_test_inv = target_scaler.inverse_transform(y_test)
    y_pred_inv = target_scaler.inverse_transform(y_pred)
    
    # Calculate metrics
    mse = mean_squared_error(y_test_inv, y_pred_inv)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)
    
    # Calculate MAPE
    mape = np.mean(np.abs((y_test_inv - y_pred_inv) / y_test_inv)) * 100
    
    print(f"\nPerformance metrics for Stock N:")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"R^2: {r2:.2f}")
    
    # Plot actual vs predicted
    plt.figure(figsize=(12, 6))
    plt.plot(y_test_inv, label='Actual')
    plt.plot(y_pred_inv, label='Predicted')
    plt.title('Stock N - Actual vs Predicted')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    
    # Save plot
    plot_path = os.path.join(save_dir, f'stock_N_test_predictions_{timestamp}.png')
    plt.savefig(plot_path)
    plt.close()
    
    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'r2': r2
    }

# Function to generate predictions for 2023
def generate_2023_predictions(model, feature_scaler, target_scaler, latest_data, feature_cols):
    """
    Generate predictions for all trading days in 2023
    
    Parameters:
    model: Trained LSTM model
    feature_scaler: Scaler for features
    target_scaler: Scaler for target
    latest_data (DataFrame): Latest available data
    feature_cols (list): Feature columns
    
    Returns:
    DataFrame: Predictions for 2023
    """
    # Generate all trading days for 2023
    def generate_2023_trading_days():
        """Generate all trading days (Mon-Fri) for 2023"""
        start_date = pd.Timestamp('2023-01-01')
        end_date = pd.Timestamp('2023-12-31')
        
        # Generate all days
        all_days = pd.date_range(start=start_date, end=end_date, freq='D')
        
        # Filter to only include weekdays (Monday=0, Sunday=6)
        trading_days = [day for day in all_days if day.weekday() < 5]
        
        # US Market Holidays 2023
        holidays_2023 = [
            '2023-01-02',  # New Year's Day (observed)
            '2023-01-16',  # Martin Luther King Jr. Day
            '2023-02-20',  # Presidents' Day
            '2023-04-07',  # Good Friday
            '2023-05-29',  # Memorial Day
            '2023-06-19',  # Juneteenth
            '2023-07-04',  # Independence Day
            '2023-09-04',  # Labor Day
            '2023-11-23',  # Thanksgiving Day
            '2023-12-25',  # Christmas Day
        ]
        
        # Remove holidays
        trading_days = [day for day in trading_days if day.strftime('%Y-%m-%d') not in holidays_2023]
        
        return trading_days
    
    trading_days = generate_2023_trading_days()
    print(f"Generated {len(trading_days)} trading days for 2023")
    
    # Get the latest sequence from the data
    extended_data = latest_data.copy()
    
    # Initialize predictions
    predictions = []
    
    # For each trading day
    for i, future_date in enumerate(trading_days):
        print(f"Predicting for {future_date.strftime('%Y-%m-%d')} ({i+1}/{len(trading_days)})", end='\r')
        
        # Get the latest sequence
        latest_sequence = extended_data.tail(sequence_length)
        
        # Extract features
        X = latest_sequence[feature_cols].values
        
        # Scale features
        X_scaled = feature_scaler.transform(X)
        
        # Reshape for LSTM [samples, time steps, features]
        X_scaled = X_scaled.reshape(1, sequence_length, len(feature_cols))
        
        # Make prediction
        y_pred_scaled = model.predict(X_scaled, verbose=0)
        
        # Inverse transform prediction
        y_pred = target_scaler.inverse_transform(y_pred_scaled)[0, 0]
        
        # Store prediction
        predictions.append({
            'Date': future_date,
            'Close': y_pred
        })
        
        # Create a new row with the prediction
        new_row = {
            'Date': future_date,
            'Close': y_pred,
            'Stock_ID': 'Stock N'
        }
        
        # Set default values for other columns
        for col in extended_data.columns:
            if col not in new_row:
                if col in ['Year', 'Month', 'Day']:
                    # Extract date components
                    if col == 'Year':
                        new_row[col] = future_date.year
                    elif col == 'Month':
                        new_row[col] = future_date.month
                    elif col == 'Day':
                        new_row[col] = future_date.day
                elif col == 'Day_of_Week':
                    new_row[col] = future_date.weekday()
                elif col == 'Quarter':
                    new_row[col] = (future_date.month - 1) // 3 + 1
                elif col == 'time_idx':
                    new_row[col] = extended_data['time_idx'].iloc[-1] + 1
                elif col in ['Open', 'High', 'Low']:
                    # For price columns, use the predicted Close
                    new_row[col] = y_pred
                elif col == 'Volume':
                    # For Volume, use the average of last 5 days
                    new_row[col] = extended_data.tail(5)['Volume'].mean()
                else:
                    # For other columns, use the last value
                    new_row[col] = extended_data[col].iloc[-1]
        
        # Append the new row
        extended_data = pd.concat([extended_data, pd.DataFrame([new_row])], ignore_index=True)
    
    # Create DataFrame with predictions
    predictions_df = pd.DataFrame(predictions)
    
    # Save predictions to CSV
    csv_path = os.path.join(save_dir, f'stock_N_predictions_2023_{timestamp}.csv')
    predictions_df.to_csv(csv_path, index=False)
    print(f"\nPredictions saved to {csv_path}")
    
    return predictions_df

# Main execution
if __name__ == "__main__":
    print("Starting LSTM model training for Stock N...")
    
    # Prepare data
    X_train, y_train, X_val, y_val, X_test, y_test, scalers = prepare_data(
        stock_n_data, feature_cols, target_col, sequence_length, prediction_days)
    
    feature_scaler, target_scaler = scalers
    
    # Save scalers
    joblib.dump(scalers, scaler_path)
    print(f"Scalers saved to: {scaler_path}")
    
    # Build and train model
    print("Training LSTM model for Stock N...")
    model, history = build_lstm_model(X_train, y_train, X_val, y_val, model_path)
    
    # Save training history
    history_df = pd.DataFrame(history.history)
    history_path = os.path.join(save_dir, f'stock_N_training_history_{timestamp}.csv')
    history_df.to_csv(history_path, index=False)
    
    # Plot training history
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Stock N Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    loss_plot_path = os.path.join(save_dir, f'stock_N_training_loss_{timestamp}.png')
    plt.savefig(loss_plot_path)
    plt.close()
    
    # Evaluate model
    print("Evaluating model...")
    metrics = evaluate_model(model, X_test, y_test, target_scaler)
    
    # Generate predictions for 2023
    print("Generating predictions for 2023...")
    predictions_df = generate_2023_predictions(
        model, feature_scaler, target_scaler, stock_n_data, feature_cols)
    
    print("\nModel training and prediction for Stock N complete!")
    print(f"Model saved to: {model_path}")
    print(f"Predictions saved to: {os.path.join(save_dir, f'stock_N_predictions_2023_{timestamp}.csv')}")
    
    # Add predictions to the final Excel file
    final_excel_path = '/Users/admin/Downloads/final_submission_predictions_2023.xlsx'
    
    try:
        # Load existing Excel file
        book = pd.ExcelFile(final_excel_path)
        with pd.ExcelWriter(final_excel_path, engine='openpyxl', mode='a') as writer:
            # Format date as string before writing
            predictions_df['Date'] = predictions_df['Date'].dt.strftime('%Y-%m-%d')
            predictions_df.to_excel(writer, sheet_name='Stock N', index=False)
        print(f"Stock N predictions added to {final_excel_path}")
    except FileNotFoundError:
        # If file doesn't exist, create a new one with just Stock N
        with pd.ExcelWriter(final_excel_path, engine='openpyxl') as writer:
            predictions_df['Date'] = predictions_df['Date'].dt.strftime('%Y-%m-%d')
            predictions_df.to_excel(writer, sheet_name='Stock N', index=False)
        print(f"Created new Excel file with Stock N predictions at {final_excel_path}")

Loading data...
Stock N data points: 116
Date range: 2022-07-18 00:00:00 to 2022-12-30 00:00:00
Missing values in Stock N:
Series([], dtype: int64)
All missing values in Stock N data have been handled.
Using 22 features for Stock N model
Starting LSTM model training for Stock N...
Training sequences: 84
Validation sequences: 10
Testing sequences: 12
Scalers saved to: /Users/admin/Downloads/stock_N_scalers_20250327_144425.pkl
Training LSTM model for Stock N...


2025-03-27 14:44:26.074666: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-03-27 14:44:26.074846: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-03-27 14:44:26.074852: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-03-27 14:44:26.075189: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-27 14:44:26.075206: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefe

Epoch 1/200


2025-03-27 14:44:29.023178: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-03-27 14:44:29.029979: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 0.1110
Epoch 1: val_loss improved from inf to 0.07369, saving model to /Users/admin/Downloads/lstm_stock_N_model_20250327_144425.h5


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0113 - val_loss: 0.0504
Epoch 6/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0096
Epoch 6: val_loss did not improve from 0.05040
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0100 - val_loss: 0.0753
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0103
Epoch 7: val_loss improved from 0.05040 to 0.04669, saving model to /Users/admin/Downloads/lstm_stock_N_model_20250327_144425.h5


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0101 - val_loss: 0.0467
Epoch 8/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0122
Epoch 8: val_loss did not improve from 0.04669
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0120 - val_loss: 0.0555
Epoch 9/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0096
Epoch 9: val_loss did not improve from 0.04669
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0098 - val_loss: 0.0682
Epoch 10/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0076
Epoch 10: val_loss did not improve from 0.04669
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0076 - val_loss: 0.0578
Epoch 11/200
 9/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0065
Epoch 11: val_loss did not improve from 0.04669
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0068 - val_loss: 0.0565
Epoch 12/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0070
Epoch 12: val_loss did not improve from 0.04669
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0071

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0061 - val_loss: 0.0450
Epoch 15/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0092
Epoch 15: val_loss did not improve from 0.04500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0091 - val_loss: 0.0645
Epoch 16/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0069
Epoch 16: val_loss improved from 0.04500 to 0.03683, saving model to /Users/admin/Downloads/lstm_stock_N_model_20250327_144425.h5


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0070 - val_loss: 0.0368
Epoch 17/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0099
Epoch 17: val_loss did not improve from 0.03683
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0096 - val_loss: 0.0589
Epoch 18/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0066
Epoch 18: val_loss did not improve from 0.03683
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0066 - val_loss: 0.0532
Epoch 19/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0049
Epoch 19: val_loss did not improve from 0.03683
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0051 - val_loss: 0.0407
Epoch 20/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0042
Epoch 20: val_loss did not improve from 0.03683
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0044 - val_loss: 0.0495
Epoch 21/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0103
Epoch 21: val_loss improved from 0.03683 to 0.02904, saving model to /Users/admin/Downloads/lstm_s

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0100 - val_loss: 0.0290
Epoch 22/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0065
Epoch 22: val_loss did not improve from 0.02904
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0066 - val_loss: 0.0637
Epoch 23/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0081
Epoch 23: val_loss did not improve from 0.02904
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0079 - val_loss: 0.0326
Epoch 24/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0054
Epoch 24: val_loss did not improve from 0.02904
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0054 - val_loss: 0.0360
Epoch 25/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0069
Epoch 25: val_loss did not improve from 0.02904
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0071 - val_loss: 0.0418
Epoch 26/200
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0052
Epoch 26: val_loss did not improve from 0.02904
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.